<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# Проект «Анализ вакансий из HeadHunter»
   

In [7]:
# импорт необходимых библиотек, требующихся для выполнения всех заданий
# выключение предупреждений (предупреждение о том, что pandas поддерживает только SQLAlchemy connectable (engine/connection) или URL-адрес строки базы данных или подключение sqlite3 DBAPI2)

import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')

In [9]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

## 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в базе (вакансии находятся в таблице `vacancies`).

In [10]:
# SQL-запрос для подсчета вакансий
query_3_1 = f'''SELECT COUNT(*) AS total_vacancies
                  FROM vacancies;
               '''

In [11]:
# Выполняем запрос и получаем результат в виде DataFrame
df_vacancies_count = pd.read_sql_query(query_3_1, connection)

# Выводим результат
print(df_vacancies_count)

   total_vacancies
0            49197


2. Напишите запрос, который посчитает количество работодателей (таблица `employers`).

In [12]:
# SQL-запрос для подсчета работодателей
query_3_2 = f'''SELECT COUNT(*) AS total_employers
                           FROM employers;'''


In [13]:
# Выполняем запрос и получаем результат в виде DataFrame
df_employers_count = pd.read_sql_query(query_3_2, connection)

# Выводим результат
print(df_employers_count)

   total_employers
0            23501


3. Посчитайте с помощью запроса количество регионов (таблица `areas`).

In [14]:
# SQL-запрос для подсчета регионов
query_3_3 = f'''SELECT COUNT(*) AS total_areas
                       FROM areas;'''

In [15]:
# Выполняем запрос и получаем результат в виде DataFrame
df_areas_count = pd.read_sql_query(query_3_3, connection)

# Выводим результат
print(df_areas_count)

   total_areas
0         1362


4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица `industries`).

In [16]:
# SQL-запрос для подсчета сфер деятельности
query_3_4 = f'''SELECT COUNT(*) AS total_industries
                            FROM industries;'''

In [17]:
# Выполняем запрос и получаем результат в виде DataFrame
df_industries_count = pd.read_sql_query(query_3_4, connection)

# Выводим результат
print(df_industries_count)

   total_industries
0               294


##На основе предоставленных данных можно сделать следующие выводы:

Общее количество вакансий: 49 197. Это значительное количество вакансий, что может указывать на активный рынок труда и разнообразие предложений по трудоустройству.

Количество работодателей: 23 501. Наличие такого количества работодателей свидетельствует о высоком уровне конкуренции среди компаний, предлагающих вакансии. Это также может означать, что существует множество различных отраслей и сфер деятельности.

Количество регионов: 1 362. Это указывает на широкое географическое покрытие вакансий. Поскольку вакансии представлены в таком количестве регионов, это дает соискателям возможность выбора и увеличивает вероятность нахождения подходящей работы в их местности.

Количество сфер деятельности: 294. Разнообразие сфер деятельности говорит о том, что на рынке существуют вакансии в различных отраслях, что является положительным показателем для соискателей с разными навыками и интересами.

Общий вывод:
Предоставленные данные указывают на динамичный и разнообразный рынок труда, где имеется множество вакансий от различных работодателей в различных регионах и сферах деятельности. Это создает возможности для соискателей с разным опытом и квалификацией. Однако высокое количество вакансий также может означать, что конкуренция среди соискателей может быть значительной, особенно в популярных сферах.

***

## 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (`cnt`) вакансий в каждом регионе (`area`).
Отсортируйте по количеству вакансий в порядке убывания.

In [18]:
# SQL-запрос для подсчета вакансий в каждом регионе
query_4_1 = f'''SELECT a.name AS area, COUNT(v.id) AS cnt
                                     FROM areas a
                                     LEFT JOIN vacancies v ON a.id = v.area_id
                                     GROUP BY a.name
                                     ORDER BY cnt DESC;'''

In [20]:
#Выполняем запрос и получаем результат в виде DataFrame
df_vacancies_count_by_area = pd.read_sql_query(query_4_1, connection)

# Выводим результат
print(df_vacancies_count_by_area)

                                   area   cnt
0                                Москва  5333
1                       Санкт-Петербург  2851
2                                 Минск  2112
3                           Новосибирск  2006
4                                Алматы  1892
...                                 ...   ...
1357                         Багаевская     0
1358                         Пролетарск     0
1359                       Седельниково     0
1360                             Инской     0
1361  Белоярский (Свердловская область)     0

[1362 rows x 2 columns]


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [21]:
# SQL-запрос для подсчета вакансий с заполненными полями salary_from или salary_to
query_4_2 = f'''SELECT COUNT(*) AS cnt
                                  FROM vacancies
                                  WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL;'''

In [22]:
# Выполняем запрос и получаем результат в виде DataFrame
df_vacancies_with_salary = pd.read_sql_query(query_4_2, connection)

# Выводим результат
print(df_vacancies_with_salary)

     cnt
0  24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до **целого числа**.

In [23]:
# SQL-запрос для нахождения средних значений зарплатной вилки
query_4_3 = f'''SELECT
                                CAST(AVG(salary_from) AS INT) AS average_salary_from,
                                CAST(AVG(salary_to) AS INT) AS average_salary_to
                             FROM vacancies
                             WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL;'''

In [24]:
# Выполняем запрос и получаем результат в виде DataFrame
df_average_salaries = pd.read_sql_query(query_4_3, connection)

# Выводим результат
print(df_average_salaries)

   average_salary_from  average_salary_to
0                71065             110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (`schedule`) и типа трудоустройства (`employment`), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [32]:
# SQL-запрос для подсчета количества вакансий по типам рабочего графика и трудоустройства
query_4_4 = f'''SELECT
                    schedule,
                    employment,
                    COUNT(*) AS vacancy_count
                FROM vacancies
                GROUP BY schedule, employment
                ORDER BY vacancy_count DESC;'''

In [33]:
# Выполняем запрос и получаем результат в виде DataFrame
df_vacancies_by_schedule_and_employment = pd.read_sql_query(query_4_4, connection)

# Выводим результат
print(df_vacancies_by_schedule_and_employment)

            schedule           employment  vacancy_count
0        Полный день     Полная занятость          35367
1   Удаленная работа     Полная занятость           7802
2      Гибкий график     Полная занятость           1593
3   Удаленная работа  Частичная занятость           1312
4     Сменный график     Полная занятость            940
5        Полный день           Стажировка            569
6     Вахтовый метод     Полная занятость            367
7        Полный день  Частичная занятость            347
8      Гибкий график  Частичная занятость            312
9        Полный день     Проектная работа            141
10  Удаленная работа     Проектная работа            133
11     Гибкий график           Стажировка            116
12    Сменный график  Частичная занятость            101
13  Удаленная работа           Стажировка             64
14     Гибкий график     Проектная работа             18
15    Сменный график           Стажировка             12
16    Вахтовый метод     Проект

5. Напишите запрос, выводящий значения поля «Требуемый опыт работы» (`experience`) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [34]:
# SQL-запрос для получения значений "Требуемый опыт работы" с подсчетом количества вакансий
query_4_5 = f'''SELECT
                  experience,
                  COUNT(*) AS vacancy_count
                FROM vacancies
                WHERE experience IS NOT NULL
                GROUP BY experience
                ORDER BY vacancy_count ASC;'''

In [35]:
# Выполняем запрос и получаем результат в виде DataFrame
df_experience_count = pd.read_sql_query(query_4_5, connection)

# Выводим результат
print(df_experience_count)

           experience  vacancy_count
0         Более 6 лет           1337
1           Нет опыта           7197
2       От 3 до 6 лет          14511
3  От 1 года до 3 лет          26152


##Выводы по детальному анализу вакансий

1. Географическое распределение вакансий:

Наибольшее количество вакансий сосредоточено в крупных городах. Москва (5333 вакансии) и Санкт-Петербург (2851 вакансия) лидируют по количеству предложений, что указывает на концентрацию рабочей силы и высокую активность рынка труда в этих регионах.
Другие города, такие как Минск (2112 вакансий) и Новосибирск (2006 вакансий), также демонстрируют значительное количество вакансий, однако их объемы значительно ниже, чем в столицах.

2. Зарплатные вилки:

Средняя нижняя граница зарплатной вилки составляет 71,065 рублей, в то время как верхняя граница — 110,537 рублей. Это указывает на наличие значительного диапазона зарплат, что может быть связано с различиями в опыте работы, квалификации и отрасли.

3. Тип рабочего графика и тип трудоустройства:

Наиболее популярными являются вакансии с полным рабочим днем и полной занятостью (35,367 вакансий). Это говорит о предпочтении работодателей к стандартным условиям труда.
Удаленная работа также востребована, с 7,802 вакансиями на полную занятость и 1,312 на частичную занятость, что отражает нарастающую тенденцию к гибкости в условиях труда.
Гибкий график и вахтовый метод также присутствуют, хотя их количество значительно меньше, что может свидетельствовать о специфических требованиях в определенных сферах.

4. Требуемый опыт работы:

Вакансии, требующие от 1 года до 3 лет опыта, составляют наибольшую долю (26,152 вакансий), что может указывать на высокий спрос на специалистов, имеющих некоторый опыт.
Вакансии для кандидатов без опыта (7,197 вакансий) также имеют значительное представительство, что может говорить о наличии программ стажировок или начальных позиций.
Вакансии для более опытных специалистов (от 3 до 6 лет и более 6 лет) менее распространены (14,511 и 1,337 соответственно), что может указывать на недостаток таких специалистов на рынке.

Заключение

Общий анализ вакансий показывает, что рынок труда в крупных городах активно развивается, с акцентом на полную занятость и стандартные условия работы. Тем не менее, наблюдается растущий интерес к удаленной работе. Важно также отметить, что высокий спрос на специалистов с 1-3 годами опыта может указывать на потребность компаний в квалифицированных кадрах, в то время как наличие вакансий для безопытных соискателей открывает возможности для начинающих специалистов.

***

## 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [36]:
# SQL-запрос для получения первых пяти работодателей по количеству вакансий
query_5_1 = f'''SELECT
                  employers.id,
                  employers.name,
                  COUNT(vacancies.id) AS vacancy_count
                FROM
                  employers
                LEFT JOIN
                  vacancies ON employers.id = vacancies.employer_id
                GROUP BY
                  employers.id, employers.name
                ORDER BY
                  vacancy_count DESC
                LIMIT 5;'''

In [37]:
# Выполняем запрос и получаем результат в виде DataFrame
df_top_employers = pd.read_sql_query(query_5_1, connection)

# Выводим первый и пятый места
first_employer = df_top_employers.iloc[0]
fifth_employer = df_top_employers.iloc[4]

print("Первый работодатель:")
print(first_employer)

print("\nПятый работодатель:")
print(fifth_employer)

Первый работодатель:
id                 1740
name             Яндекс
vacancy_count      1933
Name: 0, dtype: object

Пятый работодатель:
id                       39305
name             Газпром нефть
vacancy_count              331
Name: 4, dtype: object


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [39]:
# Запрос для получения количества работодателей и вакансий для каждого региона
query_5_2 = f'''
SELECT
    t_a.name AS region,
    COUNT(DISTINCT t_e.id) AS employers_cnt,
    COUNT(t_v.id) AS vacancies_cnt
FROM
    areas AS t_a
LEFT JOIN
    employers AS t_e ON t_a.id = t_e.area
LEFT JOIN
    vacancies AS t_v ON t_v.area_id = t_a.id
GROUP BY
    t_a.id, t_a.name
ORDER BY
    t_a.name;
'''

# Выполняем запрос и получаем результаты для всех регионов
df_all_regions = pd.read_sql_query(query_5_2, connection)

# Печатаем результаты для всех регионов
print("Количество работодателей и вакансий для каждого региона:")
print(df_all_regions)

# Запрос для нахождения региона с наибольшим количеством работодателей среди тех, где нет вакансий
query_5_2_1 = f'''
SELECT
    t_a.name AS region,
    COUNT(DISTINCT t_e.id) AS employers_cnt
FROM
    areas AS t_a
LEFT JOIN
    employers AS t_e ON t_a.id = t_e.area
LEFT JOIN
    vacancies AS t_v ON t_v.area_id = t_a.id
GROUP BY
    t_a.id, t_a.name
HAVING
    COUNT(t_v.id) = 0
ORDER BY
    employers_cnt DESC
LIMIT 1;
'''

Количество работодателей и вакансий для каждого региона:
                            region  employers_cnt  vacancies_cnt
0                           Абакан             13            481
1                             Абан              0              1
2                         Абатское              0              0
3                         Абдулино              0              1
4                           Абинск              2             12
...                            ...            ...            ...
1357                        Ярцево              0              2
1358                     Ясногорск              0              2
1359  Ясный (Оренбургская область)              0              0
1360                        Яшкино              0              0
1361                            Яя              0              1

[1362 rows x 3 columns]


In [43]:
# Выполняем запрос и получаем результаты для региона без вакансий
df_top_region_no_vacancies = pd.read_sql_query(query_5_2_1, connection)

# Печатаем результаты для региона без вакансий
print("\nРегион с наибольшим количеством работодателей без вакансий:")
print(df_top_region_no_vacancies)


Регион с наибольшим количеством работодателей без вакансий:
   region  employers_cnt
0  Россия            410


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [44]:
# Запрос для подсчета количества регионов для каждого работодателя
query_5_3 = f'''
SELECT
    t_e.id AS employer_id,
    t_e.name AS employer_name,
    COUNT(DISTINCT t_v.area_id) AS region_count
FROM
    employers AS t_e
LEFT JOIN
    vacancies AS t_v ON t_e.id = t_v.employer_id
GROUP BY
    t_e.id, t_e.name
ORDER BY
    region_count DESC;
'''

In [45]:
# Выполняем запрос и получаем результаты
df_employer_region_count = pd.read_sql_query(query_5_3, connection)

# Печатаем результаты
print("Количество регионов, в которых каждый работодатель публикует свои вакансии:")
print(df_employer_region_count)

Количество регионов, в которых каждый работодатель публикует свои вакансии:
       employer_id           employer_name  region_count
0             1740                  Яндекс           181
1             2748              Ростелеком           152
2          5724811              Спецремонт           116
3          5130287  Поляков Денис Иванович            88
4          3682876               ООО ЕФИН             71
...            ...                     ...           ...
23496      5287515              Но Смокинг             0
23497      2839354        МАСТЕРСКАЯ CREED             0
23498      5774578           Dzing Finance             0
23499      5775069                    КОРС             0
23500           15             Наука-Связь             0

[23501 rows x 3 columns]


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [46]:
# Запрос для подсчета количества работодателей без указанной сферы деятельности
query_5_4 = f'''
SELECT
    COUNT(DISTINCT e.id) AS employer_count
FROM
    employers AS e
LEFT JOIN
    employers_industries AS ei ON e.id = ei.employer_id  -- Соединяем по employer_id
WHERE
    ei.industry_id IS NULL;  -- Условие для выбора работодателей без указанной сферы деятельности
'''

In [47]:
# Выполняем запрос и получаем результат
df_employer_without_industry = pd.read_sql_query(query_5_4, connection)

# Печатаем результат
print("Количество работодателей, у которых не указана сфера деятельности:")
print(df_employer_without_industry)

Количество работодателей, у которых не указана сфера деятельности:
   employer_count
0            8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [48]:
# Запрос для получения названия компании, находящейся на третьем месте в алфавитном списке
query_5_5 = f'''
SELECT
    e.name  -- Предполагая, что поле с названием компании называется 'name'
FROM
    employers AS e
JOIN
    employers_industries AS ei ON e.id = ei.employer_id  -- Соединение по employer_id
GROUP BY
    e.id, e.name  -- Группируем по идентификатору и названию компании
HAVING
    COUNT(ei.industry_id) = 4  -- Условие для выбора только компаний с четырьмя сферами деятельности
ORDER BY
    e.name  -- Сортируем по названию компании в алфавитном порядке
LIMIT 1 OFFSET 2;  -- Пропускаем два первых результата и берем третий
'''

In [49]:
# Выполняем запрос и получаем результат
df_third_company = pd.read_sql_query(query_5_5, connection)

# Печатаем результат
print("Название компании, находящейся на третьем месте в алфавитном списке:")
print(df_third_company)

Название компании, находящейся на третьем месте в алфавитном списке:
   name
0  2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».


In [50]:
# Запрос для подсчета количества работодателей
query_5_6 = f'''SELECT
    COUNT(DISTINCT ei.employer_id) AS employer_count  -- Подсчитываем уникальных работодателей
FROM
    employers_industries AS ei
JOIN
    industries AS i ON ei.industry_id = i.id  -- Объединяем таблицы по industry_id
WHERE
    i.name = 'Разработка программного обеспечения';  -- Условие для выбора нужной сферы деятельности
'''

In [51]:
# Выполняем запрос и получаем результат
df_employers_with_software_development = pd.read_sql_query(query_5_6, connection)

# Печатаем результат
print("Количество работодателей с указанной сферой деятельности 'Разработка программного обеспечения':")
print(df_employers_with_software_development)

Количество работодателей с указанной сферой деятельности 'Разработка программного обеспечения':
   employer_count
0            3553


7. Для компании «Яндекс» выведите список [городов-миллионников](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8), в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку "Total" с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

    Если возникнут трудности с этим заданием, посмотрите материалы модуля 6.4 «Как получать данные из веб-источников и API».

In [52]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# URL страницы Википедии
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'

# Выполняем GET-запрос к сайту
response = requests.get(url)

# Проверяем успешность запроса
if response.status_code == 200:
    # Создаем объект BeautifulSoup для парсинга HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Находим таблицу с городами
    table = soup.find('table', {'class': 'standard sortable'})

    # Проверяем, была ли найдена таблица
    if table is not None:
        # Создаем списки для хранения данных
        numbers = []
        cities = []

        # Извлекаем строки таблицы
        for row in table.find_all('tr')[1:]:  # Пропускаем заголовок таблицы
            columns = row.find_all('td')
            if columns:
                # Извлекаем номер и название города
                numbers.append(columns[0].text.strip())
                cities.append(columns[1].text.strip())

        # Создаем DataFrame
        df = pd.DataFrame({
            'Номер': numbers,
            'Город': cities
        })

        # Выводим таблицу
        print(df)
    else:
        print("Таблица не найдена.")
else:
    print(f"Ошибка при запросе: {response.status_code}")

   Номер            Город
0      1           Москва
1      2  Санкт-Петербург
2      3      Новосибирск
3      4     Екатеринбург
4      5           Казань
5      6       Красноярск
6      7  Нижний Новгород
7      8        Челябинск
8      9              Уфа
9     10           Самара
10    11   Ростов-на-Дону
11    12        Краснодар
12    13             Омск
13    14          Воронеж
14    15            Пермь
15    16        Волгоград


In [53]:
# Преобразование списка городов в строку для SQL-запроса
city_names = ', '.join(f"'{city}'" for city in df['Город'])

# Запрос для получения количества вакансий для работодателя "Яндекс" в указанных регионах
query_5_7 = f'''
WITH VacancyCounts AS (
    SELECT
        a.name AS city_name,
        COUNT(t_v.id) AS vacancy_count
    FROM
        vacancies AS t_v
    JOIN
        employers AS t_e ON t_v.employer_id = t_e.id
    JOIN
        areas AS a ON t_v.area_id = a.id
    WHERE
        t_e.name = 'Яндекс'
        AND a.name IN ({city_names})  -- Используем city_names здесь
    GROUP BY
        a.name
)

SELECT
    city_name,
    vacancy_count
FROM
    VacancyCounts

UNION ALL

SELECT
    'Total' AS city_name,
    SUM(vacancy_count) AS vacancy_count
FROM
    VacancyCounts
ORDER BY
    vacancy_count;
'''

In [54]:
# Выполняем запрос и получаем результаты
df_vacancy_counts = pd.read_sql_query(query_5_7, connection)

# Печатаем результаты
print("Количество вакансий компании 'Яндекс' в указанных регионах:")
print(df_vacancy_counts)

Количество вакансий компании 'Яндекс' в указанных регионах:
          city_name  vacancy_count
0              Омск           21.0
1         Челябинск           22.0
2        Красноярск           23.0
3         Волгоград           24.0
4             Пермь           25.0
5            Казань           25.0
6    Ростов-на-Дону           25.0
7            Самара           26.0
8               Уфа           26.0
9         Краснодар           30.0
10          Воронеж           32.0
11      Новосибирск           35.0
12  Нижний Новгород           36.0
13     Екатеринбург           39.0
14  Санкт-Петербург           42.0
15           Москва           54.0
16            Total          485.0


##Вывод по анализу работодателей и рынка вакансий
Эти данные позволяют получить четкое представление о текущем состоянии рынка вакансий в разных регионах и распределении работодателей по сферам деятельности.
В частности, можно выделить несколько ключевых аспектов:

Региональное распределение вакансий:

Наблюдается значительная разница в количестве вакансий между различными регионами. Например, в таких крупных городах, как Москва и Санкт-Петербург, сосредоточено значительно больше вакансий по сравнению с менее населенными регионами. Это может указывать на концентрацию бизнеса и высокую активность рынка труда в крупных городах.
Регион Абакан с 481 вакансией и 13 работодателями демонстрирует, что даже в относительно небольших городах может быть активный рынок труда, хотя и с меньшей конкуренцией.

Работодатели с активными вакансиями:

Работодатели, такие как Яндекс, занимают лидирующие позиции по количеству предложений, что может свидетельствовать о высоком спросе на специалистов в IT-сфере и технологий. Это также подчеркивает важность таких компаний в экономике и на рынке труда.
Наличие работодателя, как Газпром нефть, на пятом месте показывает, что традиционные сферы, такие как энергетика, также играют значительную роль на рынке.

Работодатели без вакансий:

Обнаружение большого количества работодателей без открытых вакансий (например, 8419) может указывать на несколько факторов: возможно, они не активно ищут сотрудников в данный момент или же испытывают трудности в найме, что может быть связано с изменениями на рынке или экономической ситуацией.
Регион Россия с 410 работодателями без вакансий может означать, что в этом регионе существует потенциальный рынок работников, но работодатели не используют его, что вызывает вопросы о возможности улучшения ситуации.

Сферы деятельности:

Данные о работодателях в сфере разработки программного обеспечения (3553) подчеркивают растущую значимость технологий и цифровизации, что отражает глобальные тренды. Это может быть связано с увеличением инвестиций в IT и цифровую инфраструктуру.
Интерес к компаниям с несколькими сферами деятельности, как, например, у 2ГИС, помогает понять, какие навыки и знания могут быть востребованы в будущем, и как работники могут адаптироваться к изменяющимся требованиям рынка.

Вакансии в городах-миллионниках:

Данные о вакансиях компании «Яндекс» в городах-миллионниках показывают, что крупные компании ищут кадры не только в столицах, но и в регионах с развитой инфраструктурой и населением. Это создает возможность для трудоустройства и карьерного роста для жителей этих городов.
Общая сумма вакансий в 485 подчеркивает, что компания активно развивает свои позиции в различных регионах, что может быть признаком роста бизнеса и увеличения спроса на услуги.

Заключение

Анализ данных об работодателях и вакансиях позволяет выявить ключевые тенденции на рынке труда, определить области с высоким потенциалом для карьерного роста, а также понять, какие сферы деятельности становятся более востребованными. Такой анализ может служить основой для разработки стратегий по повышению конкурентоспособности на рынке труда, как для работодателей, так и для соискателей. Важно также учитывать, что обстоятельства могут меняться, и регулярный мониторинг данных поможет быть в курсе текущих тенденций.



***

## 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

    Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова `'data'` или `'данн'`.

    *Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [55]:
# SQL-запрос для подсчета вакансий, которые имеют отношение к данным, если в её названии содержатся слова 'data' или 'данн'
query_6_1 = f'''
SELECT COUNT(*) AS related_vacancy_count
FROM public.vacancies
WHERE LOWER(name) LIKE '%data%' OR LOWER(name) LIKE '%данн%';
'''



In [56]:
# Выполняем запрос и получаем результаты
df_related_vacancy_count = pd.read_sql_query(query_6_1, connection)

    # Вывод результата
print(df_related_vacancy_count)

   related_vacancy_count
0                   1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
    * 'data scientist'
    * 'data science'
    * 'исследователь данных'
    * 'ML' (здесь не нужно брать вакансии по HTML)
    * 'machine learning'
    * 'машинн%обучен%'

    **В следующих заданиях мы продолжим работать с вакансиями по этому условию.**

    Считаем вакансиями для специалистов уровня Junior следующие:
    + в названии есть слово “junior” **или**
    + требуемый опыт — «Нет опыта» **или**
    + тип трудоустройства — «Стажировка».


In [59]:
# количество вакансий для начинающих DS
query_6_2 = f'''select
               count(name) cnt_data_vacancies
               from vacancies
               where (name ilike '%data scientist%'
                  or name ilike '%data science%'
                  or name ilike '%исследователь данных%'
                  or name ilike '%ML%' and name not ilike '%HTML%'
                  or name ilike '%machine learning%'
                  or name ilike '%машинн%обучен%')
                  and (name ilike '%junior%'
                  or experience ilike 'Нет опыта'
                  or employment ilike 'Стажировка')
            '''

In [60]:
# результат запроса
vacancies_count_ds = pd.read_sql_query(query_6_2, connection)
    # Вывод результата
print(vacancies_count_ds)

   cnt_data_vacancies
0                  51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или Postgres?

    *Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [62]:
# количество вакансий для DS с ключевым навыком "SQL" или "postgres"
query_6_3 = f'''select
               count(name) cnt_data_vacancies
               from vacancies
               where (name ilike '%data scientist%'
                  or name ilike '%data science%'
                  or name ilike '%исследователь данных%'
                  or name ilike '%ML%' and name not ilike '%HTML%'
                  or name ilike '%machine learning%'
                  or name ilike '%машинн%обучен%')
                  and (key_skills ilike '%SQL%'
                  or key_skills ilike '%postgres%')
            '''

In [63]:
ds_vacancies_with_sql_or_postgres_count = pd.read_sql_query(query_6_3, connection)
    # Вывод результата
print(ds_vacancies_with_sql_or_postgres_count)

   cnt_data_vacancies
0                 229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

    *Это можно сделать помощью запроса, аналогичного предыдущему.*

In [64]:
# количество вакансий для DS с ключевым навыком "Python"
query_6_4 = f'''select
               count(name) cnt_data_vacancies
               from vacancies
               where (name ilike '%data scientist%'
                  or name ilike '%data science%'
                  or name ilike '%исследователь данных%'
                  or name ilike '%ML%' and name not ilike '%HTML%'
                  or name ilike '%machine learning%'
                  or name ilike '%машинн%обучен%')
                  and key_skills ilike '%Python%'
            '''

In [65]:
ds_vacancies_with_python_count = pd.read_sql_query(query_6_4, connection)
print(ds_vacancies_with_python_count)

   cnt_data_vacancies
0                 357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до **двух знаков после точки-разделителя**.

In [68]:
# среднее количесво требуемых ключевых навыков для DS
query_6_5 = f'''select
               round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1), 2) avg_key_skills
               from vacancies
               where name ilike '%data scientist%'
                  or name ilike '%data science%'
                  or name ilike '%исследователь данных%'
                  or name like '%ML%' and name not ilike '%HTML%'
                  or name ilike '%machine learning%'
                  or name ilike '%машинн%обучен%'
                  and key_skills is not null
            '''

In [69]:
average_required_skills_for_ds = pd.read_sql_query(query_6_5, connection)
print(average_required_skills_for_ds)

   avg_key_skills
0            6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля `experience`).

    При решении задачи примите во внимание следующее:
    1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
    2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
    3. Если в расчётах участвует `null`, в результате он тоже даст `null` (посмотрите, что возвращает запрос `select 1 + null`). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит `null` на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

    Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до **целого числа**.

In [70]:
# средняя зарплата для каждого уникального типа требуемого опыта
query_6_6 = f'''select
               distinct experience,
               round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)))
               from vacancies
               where (name ilike '%data scientist%'
                  or name ilike '%data science%'
                  or name ilike '%исследователь данных%'
                  or name like '%ML%' and name not ilike '%HTML%'
                  or name ilike '%machine learning%'
                  or name ilike '%машинн%обучен%')
                  and (salary_from is not null or salary_to is not null)
               group by 1
            '''

In [71]:
average_salary_by_experience_type = pd.read_sql_query(query_6_6, connection)
average_salary_by_experience_type

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


##На основе представленных данных по предметному анализу вакансий для дата-сайентистов можно сделать следующие выводы:

Общее количество вакансий: Вакансий, имеющих отношение к данным, достаточно много - 1771. Это свидетельствует о высоком спросе на специалистов в области данных.

Вакансии для начинающих: Из 1771 вакансии только 51 подходит для начинающего дата-сайентиста. Это может указывать на ограниченные возможности для входа в профессию или на высокие требования к кандидатам даже на начальном уровне.

Ключевые навыки:

Вакансий, в которых в качестве ключевого навыка указан SQL или Postgres, достаточно много - 229. Это подчеркивает важность этих технологий в области анализа данных и работы с базами данных.

Количество вакансий, требующих знание Python, составляет 357. Python остается одним из самых популярных языков программирования в сфере данных, что подтверждает его значимость для специалистов.

Среднее количество ключевых навыков: Вакансии для дата-сайентистов в среднем содержат 6,41 ключевых навыка. Это говорит о том, что работодатели ожидают от кандидатов широких знаний и навыков, что может быть вызвано многообразием задач, с которыми сталкиваются дата-сайентисты.

Средняя зарплата по опыту:

Для специалистов без опыта средняя зарплата составляет 74643.0, что может быть относительно низким для начала карьеры.

Специалисты с опытом от 1 до 3 лет могут рассчитывать на среднюю зарплату в 139675.0, что значительно выше и отражает рост заработка с увеличением опыта.

Специалисты с опытом от 3 до 6 лет получают в среднем 243115.0, что указывает на высокую ценность опытных дата-сайентистов на рынке труда.

В общем, рынок вакансий для дата-сайентистов демонстрирует значительный интерес к специалистам, однако, для начинающих возможностей немного, а требования к навыкам высоки. С увеличением опыта заработная плата значительно возрастает, что подчеркивает ценность профессионального роста в этой области.

***

## Общий вывод по проекту


На основе представленных данных можно сделать общий вывод о состоянии рынка труда, который демонстрирует активность и разнообразие в вакансиях, работодателях, регионах и сферах деятельности.

Активный рынок труда: Общее количество вакансий составляет 49 197, что указывает на высокую динамику и разнообразие предложений по трудоустройству. Это создает возможности для соискателей с различными уровнями квалификации и опытом.

Конкуренция среди работодателей: С 23 501 работодателями на рынке наблюдается высокая конкуренция, что может приводить к улучшению условий труда и зарплат для соискателей.

Широкое географическое покрытие: Вакансии представлены в 1 362 регионах, что дает соискателям возможность находить работу в различных местах, включая как крупные города, так и менее населенные регионы.

Разнообразие сфер деятельности: Наличие 294 сфер деятельности свидетельствует о многогранности рынка труда. Соискатели могут выбирать из множества направлений, что позволяет находить работу, соответствующую их интересам и навыкам.

Географическое распределение вакансий: Наибольшее количество вакансий сосредоточено в крупных городах, таких как Москва и Санкт-Петербург, что указывает на концентрацию бизнеса и высокую активность рынка труда в этих регионах.

Зарплатные вилки: Средняя нижняя граница зарплаты составляет 71,065 рублей, а верхняя — 110,537 рублей, что свидетельствует о значительном диапазоне зарплат в зависимости от опыта, квалификации и отрасли.

Тип занятости: Вакансии с полным рабочим днем и полной занятостью преобладают, однако наблюдается растущий интерес к удаленной работе, что отражает изменяющиеся требования соискателей и работодателей.

Требования к опыту: Наибольшее количество вакансий требует от 1 до 3 лет опыта. Это может указывать на высокий спрос на специалистов с определенным опытом, в то время как вакансии для начинающих также присутствуют, создавая возможности для начального входа в профессию.

В заключение, рынок труда показывает высокую активность и разнообразие, предлагая множество вакансий от различных работодателей и в различных регионах. Однако высокая конкуренция может создать определенные сложности для соискателей, особенно для начинающих специалистов. Регулярный мониторинг данных о рынке труда поможет оставаться в курсе текущих тенденций и адаптироваться к изменяющимся условиям.

# Дополнительные исследования

1. Посчитаем количество вакансий для каждого работодателя в сфере DS.

In [72]:
# количесиво вакансий для DS на каждого работодателя
query_1 = '''with t as
            (select
            e.name employer,
            count(*) data_vacancies
            from vacancies v
            join employers e on e.id = v.employer_id
            where v.name ilike '%data scientist%'
               or v.name ilike '%data science%'
               or v.name ilike '%исследователь данных%'
               or v.name ilike '%ML%' and v.name not ilike '%HTML%'
               or v.name ilike '%machine learning%'
               or v.name ilike '%машинн%обучен%'
            group by e.name
            order by 2 desc
            )

            select
            t.employer,
            t.data_vacancies
            from t
            union all
            select
            'total',
            sum(t.data_vacancies)
            from t
         '''

In [73]:
# результат запроса
df = pd.read_sql_query(query_1, connection)
df

,employer,data_vacancies
0,СБЕР,39.0
1,Bell Integrator,25.0
2,Банк ВТБ (ПАО),18.0
3,VK,15.0
4,Positive Technologies,11.0
...,...,...
258,А1 (Унитарное предприятие А1),1.0
259,СТ,1.0
260,АБСОЛЮТ БАНК,1.0
261,Gurtam,1.0


2. Выведем среднее количество вакансий на каждого работодателя.

In [74]:
# среднее количесиво вакансий для DS на каждого работодателя
query_2 = '''with t as
            (select
            e.name employer,
            count(*) data_vacancies
            from vacancies v
            join employers e on e.id = v.employer_id
            where v.name ilike '%data scientist%'
               or v.name ilike '%data science%'
               or v.name ilike '%исследователь данных%'
               or v.name ilike '%ML%' and v.name not ilike '%HTML%'
               or v.name ilike '%machine learning%'
               or v.name ilike '%машинн%обучен%'
            group by e.name
            order by 2 desc
            )

            select
            round(avg(t.data_vacancies), 2) avg_data_vacancies
            from t
         '''

In [75]:
# результат запроса
df = pd.read_sql_query(query_2, connection)
df

,avg_data_vacancies
0,2.05


3. Посчитаем среднюю зарплату для каждого уникального типа требуемого опыта работы по всем вакансиям, чтобы сравнить со значениями зарплат для DS

In [77]:
# средняя зарплата для каждого уникального типа требуемого опыта
query_3 = '''select
               distinct experience,
               round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)))
               from vacancies
               group by 1
            '''

In [78]:
# результат запроса
df = pd.read_sql_query(query_3, connection)
df

,experience,round
0,Более 6 лет,166256.0
1,Нет опыта,40472.0
2,От 1 года до 3 лет,76541.0
3,От 3 до 6 лет,133211.0


In [79]:
# закрываем соединение
connection.close()

## На основе представленных данных можно сделать следующие выводы по вакансиям в сфере дата-сайентистов (DS):

Количество вакансий по работодателям:

Всего в сфере DS имеется 536 вакансий, распределенных среди различных работодателей. Наиболее активные работодатели включают СБЕР (39 вакансий), Bell Integrator (25 вакансий) и Банк ВТБ (18 вакансий). Это может указывать на высокую заинтересованность этих компаний в привлечении специалистов в области данных.

Среднее количество вакансий на работодателя:

В среднем на каждого работодателя приходится 2,05 вакансии. Это значение говорит о том, что большинство работодателей предлагают ограниченное количество вакансий в данной сфере, что может быть связано с узкой специализацией или требованиями к кандидатам.

Сравнение зарплат по опыту:

Средняя зарплата для всех вакансий по различным уровням опыта составляет:

Нет опыта: 40,472 рублей
От 1 года до 3 лет: 76,541 рублей
От 3 до 6 лет: 133,211 рублей
Более 6 лет: 166,256 рублей

В то же время, средняя зарплата для вакансий в сфере DS значительно выше:

Нет опыта: 74,643 рублей
От 1 года до 3 лет: 139,675 рублей
От 3 до 6 лет: 243,115 рублей

Это свидетельствует о том, что вакансии в сфере DS предлагают более конкурентоспособные условия оплаты труда по сравнению с общим рынком.

Выводы: Рынок вакансий для дата-сайентистов демонстрирует активный спрос на специалистов, особенно со стороны крупных компаний. Зарплаты в данной области значительно превышают средние значения для других сфер, что делает профессию привлекательной для соискателей. Однако, среднее количество вакансий на работодателя относительно невелико, что может указывать на узкую специализацию и высокие требования к кандидатам. Это также может означать, что несмотря на общее количество вакансий, конкуренция за позиции может быть высокой.